In [1]:
import pandas as pd

In [2]:
df_diag = pd.DataFrame()
temp = pd.read_csv('../data/DIAGNOSES_ICD.csv', iterator=True, chunksize=1000)
df_diag = pd.concat(temp, ignore_index=True)
print(df_diag.head(5))


   ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
0    1297         109   172335      1.0     40301
1    1298         109   172335      2.0       486
2    1299         109   172335      3.0     58281
3    1300         109   172335      4.0      5855
4    1301         109   172335      5.0      4254


In [3]:
df_notes = pd.DataFrame()
temp = pd.read_csv('../data/NOTEEVENTS.csv', iterator=True, chunksize=1000)
df_notes = pd.concat(temp, ignore_index=True)
print(df_notes.head(5))

   ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0     174       22532  167853.0  2151-08-04       NaN       NaN   
1     175       13702  107527.0  2118-06-14       NaN       NaN   
2     176       13702  167118.0  2119-05-25       NaN       NaN   
3     177       13702  196489.0  2124-08-18       NaN       NaN   
4     178       26880  135453.0  2162-03-25       NaN       NaN   

            CATEGORY DESCRIPTION  CGID  ISERROR  \
0  Discharge summary      Report   NaN      NaN   
1  Discharge summary      Report   NaN      NaN   
2  Discharge summary      Report   NaN      NaN   
3  Discharge summary      Report   NaN      NaN   
4  Discharge summary      Report   NaN      NaN   

                                                TEXT  
0  Admission Date:  [**2151-7-16**]       Dischar...  
1  Admission Date:  [**2118-6-2**]       Discharg...  
2  Admission Date:  [**2119-5-4**]              D...  
3  Admission Date:  [**2124-7-21**]              ...  
4  Admission Da

In [4]:
print(df_notes.shape)
print(df_diag.shape)

(2083180, 11)
(651047, 5)


In [5]:

# Unique subject IDs with Diabetes
# df_sub_id_diabetes = df_diag[df_diag['ICD9_CODE'].str.startswith("250", na = False)]['SUBJECT_ID'].to_frame()
df_sub_id_diabetes = df_diag[df_diag['ICD9_CODE'].str.startswith("250", na = False)]['HADM_ID'].to_frame()


df_sub_id_diabetes = df_sub_id_diabetes.reset_index(drop=True)
df_sub_id_diabetes = df_sub_id_diabetes.drop_duplicates()
print(df_sub_id_diabetes.shape)
print(df_sub_id_diabetes.head())

(14222, 1)
   HADM_ID
0   140784
1   164853
2   195632
3   113323
4   198214


In [6]:
# df_diag_diabetes = pd.merge(df_diag, df_sub_id_diabetes, on=['SUBJECT_ID'], how='inner')
df_diag_diabetes = pd.merge(df_diag, df_sub_id_diabetes, on=['HADM_ID'], how='inner')

print(df_diag_diabetes.shape)
print(df_diag_diabetes.head(2))



(199964, 5)
   ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
0    1523         117   140784      1.0      5715
1    1524         117   140784      2.0      7895


In [7]:
# df_notes_diabetes = pd.merge(df_notes, df_sub_id_diabetes, on=['SUBJECT_ID'], how='inner')
df_notes_diabetes = pd.merge(df_notes, df_sub_id_diabetes, on=['HADM_ID'], how='inner')

print(df_notes_diabetes.shape)
print(df_notes_diabetes.head(2))

(406203, 11)
   ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0     184       28063  121936.0  2125-02-16       NaN       NaN   
1   59801       28063  121936.0  2125-02-12       NaN       NaN   

            CATEGORY DESCRIPTION  CGID  ISERROR  \
0  Discharge summary      Report   NaN      NaN   
1               Echo      Report   NaN      NaN   

                                                TEXT  
0  Admission Date:  [**2125-2-9**]              D...  
1  PATIENT/TEST INFORMATION:\nIndication: Aortic ...  


In [8]:
df_notes_diabetes['TEXT'] = df_notes_diabetes['TEXT'].replace(r"[^\w\s']", ' ', regex=True).replace('[\d]', 'd',regex=True)
# df_notes_diabetes['TEXT'] = df_notes_diabetes['TEXT']
df_notes_diabetes['TEXT'].head(5)

0    Admission Date      dddd d d                 D...
1    PATIENT TEST INFORMATION \nIndication  Aortic ...
2    PATIENT TEST INFORMATION \nIndication   Aortic...
3    Sinus rhythm   Frequent atrial premature beats...
4    Rhythm is most likely sinus rhythm with freque...
Name: TEXT, dtype: object

In [9]:
df_notes_diabetes['TEXT'] = df_notes_diabetes['TEXT'].str.lower()

df_notes_diabetes.head(5)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,184,28063,121936.0,2125-02-16,NaN,NaN,Discharge summary,Report,NaN,NaN,admission date dddd d d d...
1,59801,28063,121936.0,2125-02-12,NaN,NaN,Echo,Report,NaN,NaN,patient test information \nindication aortic ...
2,59802,28063,121936.0,2125-02-11,NaN,NaN,Echo,Report,NaN,NaN,patient test information \nindication aortic...
3,105548,28063,121936.0,2125-02-10,NaN,NaN,ECG,Report,NaN,NaN,sinus rhythm frequent atrial premature beats...
4,105549,28063,121936.0,2125-02-09,NaN,NaN,ECG,Report,NaN,NaN,rhythm is most likely sinus rhythm with freque...


In [10]:
df_notes_diabetes['TOKENS'] = df_notes_diabetes['TEXT'].str.split()
print(df_notes_diabetes.head(5))

   ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0     184       28063  121936.0  2125-02-16       NaN       NaN   
1   59801       28063  121936.0  2125-02-12       NaN       NaN   
2   59802       28063  121936.0  2125-02-11       NaN       NaN   
3  105548       28063  121936.0  2125-02-10       NaN       NaN   
4  105549       28063  121936.0  2125-02-09       NaN       NaN   

            CATEGORY DESCRIPTION  CGID  ISERROR  \
0  Discharge summary      Report   NaN      NaN   
1               Echo      Report   NaN      NaN   
2               Echo      Report   NaN      NaN   
3                ECG      Report   NaN      NaN   
4                ECG      Report   NaN      NaN   

                                                TEXT  \
0  admission date      dddd d d                 d...   
1  patient test information \nindication  aortic ...   
2  patient test information \nindication   aortic...   
3  sinus rhythm   frequent atrial premature beats...   
4  rhythm 

In [11]:
print(len(df_notes_diabetes['TOKENS'][7]))

20


In [12]:
# from collections import Counter

# def wordListToFreqDict(wordFreqDict, wordlist):
#     wordfreq = [wordlist.count(p) for p in wordlist]
#     tempDict = dict(list(zip(wordlist,wordfreq)))
#     z = dict(Counter(wordFreqDict)+Counter(tempDict))
#     return z

In [13]:
# print(wordListToFreqDict({}, df_notes_diabetes['TOKENS'][7]))

In [56]:
wordCorpus = {}
word_freq_dict_list = []
max_limit = 1000000

for idx, tokens in enumerate(df_notes_diabetes['TOKENS']):
  wordfreq = [tokens.count(p) for p in tokens]
  tempDict = dict(list(zip(tokens,wordfreq)))
  word_freq_dict_list.append(tempDict)
  if idx % 10000 == 0:
    print('notes processed: ', idx)
  if idx == max_limit:
    break

notes processed:  0
notes processed:  10000
notes processed:  20000
notes processed:  30000
notes processed:  40000
notes processed:  50000
notes processed:  60000
notes processed:  70000
notes processed:  80000
notes processed:  90000
notes processed:  100000
notes processed:  110000
notes processed:  120000
notes processed:  130000
notes processed:  140000
notes processed:  150000
notes processed:  160000
notes processed:  170000
notes processed:  180000
notes processed:  190000
notes processed:  200000
notes processed:  210000
notes processed:  220000
notes processed:  230000
notes processed:  240000
notes processed:  250000
notes processed:  260000
notes processed:  270000
notes processed:  280000
notes processed:  290000
notes processed:  300000
notes processed:  310000
notes processed:  320000
notes processed:  330000
notes processed:  340000
notes processed:  350000
notes processed:  360000
notes processed:  370000
notes processed:  380000
notes processed:  390000
notes processe

In [58]:
# write to file - word frequency for each report

# import pickle5 as pkl
# pkl.dump( word_freq_dict_list, open( "../data/wordFreq.p", "wb" ) )
print(len(word_freq_dict_list))

406203


In [59]:
# read word frequency for each report
word_freq_dict_list = []
word_freq_dict_list = pkl.load(open( "../data/wordFreq.p",'rb'))
print(len(word_freq_dict_list))

406203


In [71]:
len_200 = 0
len_500 = 0
len_1000 = 0
len_2000 = 0
len_rest = 0

for dict1 in word_freq_dict_list:
  lent = len(dict1)
  if lent > 0 and lent <= 200:
    len_200 = len_200 + 1
  elif lent > 200 and lent <= 500:
    len_500 = len_500 + 1
  elif lent > 500 and lent <= 1000:
    len_1000 = len_1000 + 1
  elif lent > 1000 and lent <= 2000:
    len_2000 = len_2000 + 1
  else:
    len_rest = len_rest + 1

print(len_200, len_500, len_1000, len_2000, len_rest)


303645 80841 20852 852 13


In [77]:
wordCorpus = {}

def createWordCorpus(dictList, min_len, max_len, wordCorpus):
  for idx, dict1 in enumerate(dictList):
    lent = len(dict1)
    if lent > min_len and lent <= max_len:
      for key, value in dict1.items():
        if key in wordCorpus:
          wordCorpus[key] = wordCorpus[key] + dict1[key]
        else:
          wordCorpus[key] = dict1[key]
  return wordCorpus

wordCorpus = createWordCorpus(word_freq_dict_list, 0, 200, wordCorpus)
print(len(wordCorpus))

wordCorpus = createWordCorpus(word_freq_dict_list, 200, 500, wordCorpus)
print(len(wordCorpus))

wordCorpus = createWordCorpus(word_freq_dict_list, 500, 1000, wordCorpus)
print(len(wordCorpus))

wordCorpus = createWordCorpus(word_freq_dict_list, 1000, 100000, wordCorpus)
print(len(wordCorpus))


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
98664
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
136184
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
160116
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000


In [80]:
print(len(wordCorpus))

# write to file - word corpus with frequency - overall
# pkl.dump( wordCorpus, open( "../data/wordCorpus.p", "wb" ) )

# read from file - word corpus with frequency - overall

# wordCorpus = pkl.load(open( "../data/wordCorpus.p",'rb'))
# print(len(wordCorpus))


163167
163167


In [82]:
tokens_freq_lt_5 = {}
tokens_freq_gt_5 = {}

freq_limit = 5
for key, value in wordCorpus.items():
  if value < freq_limit:
    tokens_freq_lt_5[key] = value
  else:
    tokens_freq_gt_5[key] = value

print(len(tokens_freq_lt_5), len(tokens_freq_gt_5))

109863 53304


In [51]:
import Levenshtein as lev

def closestMatch(candidateToken, wordList):
  min_dist = 99
  similar_word = ''
  for word in wordList:
    dist = lev.distance(candidateToken, word)
    if dist <= min_dist:
      min_dist = dist
      similar_word = word
  return candidateToken, similar_word, min_dist

In [104]:
# token mapping

token_map = {}

ii = 0
for ct in tokens_freq_lt_5:
  candidateToken, similar_word, min_dist = closestMatch(ct, tokens_freq_gt_5)
  token_map[ct] = similar_word
  if ii % 10000 == 0:
    print(ii, candidateToken, similar_word, min_dist)
  ii = ii + 1


0 inferolaterl inferolateral 1
10000 exacerbatiopn exacerbation 1
20000 ruffed cuffed 1
30000 lsast llast 1
40000 coupiuos copiuos 1
50000 swishing wishing 1
60000 cvvhad cvvhd 1
70000 professed progessed 1
80000 behelpful unhelpful 2
90000 elevtive elective 1
100000 normoreflexic hyporeflexic 4


In [113]:
print(len(token_map))

# write to file - word corpus with frequency - overall
# pkl.dump( token_map, open( "../data/tokenMap.p", "wb" ) )

# read from file - word corpus with frequency - overall

# token_map = pkl.load(open( "../data/tokenMap.p",'rb'))
print(len(token_map))
print(token_map['exacerbatiopn'])

109863
109863
exacerbation


In [117]:
def map_tokens(tokens):
  n_tokens = []
  for token in tokens:
    if token in tokens_freq_lt_5:
      n_tokens.append(token_map[token])
    else:
      n_tokens.append(token)
  return n_tokens

In [118]:

df_notes_diabetes['NTOKENS'] = " "

# n_tokens = []
# for idx, tokens in enumerate(df_notes_diabetes['TOKENS']):
#   n_tokens = []
#   for token in tokens:
#     if token in tokens_freq_lt_5:
#       n_tokens.append(token_map[token])
#     else:
#       n_tokens.append(token)


#   df_notes_diabetes['NTOKENS'][idx] = n_tokens
#   if idx % 10000 == 0:
#     print(idx)
#   if idx == 1000000:
#     break

df_notes_diabetes['NTOKENS'] = df_notes_diabetes['TOKENS'].apply(map_tokens)

print(df_notes_diabetes['NTOKENS'].head(10))

0    [admission, date, dddd, d, d, discharge, date,...
1    [patient, test, information, indication, aorti...
2    [patient, test, information, indication, aorti...
3    [sinus, rhythm, frequent, atrial, premature, b...
4    [rhythm, is, most, likely, sinus, rhythm, with...
5    [atrial, fibrillation, intraventricular, condu...
6    [probable, atrial, fibrillation, left, bundle,...
7    [atrial, fibrillation, versus, wandering, atri...
8    [probable, atrial, fibrillation, with, moderat...
9    [most, likely, sinus, tachycardia, with, atria...
Name: NTOKENS, dtype: object


In [119]:
print(df_notes_diabetes['NTOKENS'].tail(10))

406193    [continued, sugar, came, up, to, ddd, md, awar...
406194    [rn, progress, note, dddd, d, dd, dddam, neuro...
406195    [rn, admit, and, progress, note, dddd, d, dd, ...
406196    [pt, admitted, direct, to, unit, by, ambulance...
406197    [msicu, nursing, progress, note, see, careview...
406198    [s, micu, nursing, progress, note, pt, started...
406199    [nursing, progress, notes, from, dddd, to, ddd...
406200    [nursing, progress, note, dddd, dddd, please, ...
406201    [dddd, dddd, rn, notes, micu, dd, y, o, f, pre...
406202    [tsicu, progress, note, dddd, dddd, ddyo, fema...
Name: NTOKENS, dtype: object


In [120]:
# write to file
# df_notes_diabetes.to_pickle("../data/notes_diabetes.p")

In [124]:
df_notes_diabetes['NTOKENS_LEN'] = df_notes_diabetes['NTOKENS'].apply(lambda x: len(x))

In [135]:
# Filtering based on # tokens - GT 9 and LE 2200
temp = df_notes_diabetes[(df_notes_diabetes['NTOKENS_LEN'] > 9) & (df_notes_diabetes['NTOKENS_LEN'] <= 2200)]
print(temp.shape)

temp1 = df_notes_diabetes[(df_notes_diabetes['NTOKENS_LEN'] <= 9) | (df_notes_diabetes['NTOKENS_LEN'] > 2200)]
print(temp1.shape)

(399629, 14)
(6574, 14)


In [136]:
print(temp1.head(5))

     ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0       184       28063  121936.0  2125-02-16       NaN       NaN   
66      193       53151  150192.0  2182-11-03       NaN       NaN   
86      195       22180  162436.0  2134-03-09       NaN       NaN   
140     198       22180  190828.0  2139-12-04       NaN       NaN   
148  105853       22180  133797.0  2139-12-26       NaN       NaN   

              CATEGORY DESCRIPTION  CGID  ISERROR  \
0    Discharge summary      Report   NaN      NaN   
66   Discharge summary      Report   NaN      NaN   
86   Discharge summary      Report   NaN      NaN   
140  Discharge summary      Report   NaN      NaN   
148                ECG      Report   NaN      NaN   

                                                  TEXT  \
0    admission date      dddd d d                 d...   
66   admission date      dddd dd dd                ...   
86   admission date      dddd d dd                 ...   
140  admission date      dddd dd d

In [138]:
print(df_diag_diabetes[(df_diag_diabetes['HADM_ID'] == 121936)])

       ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
98550  317369       28063   121936      1.0     42843
98551  317370       28063   121936      2.0     41071
98552  317371       28063   121936      3.0      5990
98553  317372       28063   121936      4.0      4275
98554  317373       28063   121936      5.0      5849
98555  317374       28063   121936      6.0      5070
98556  317375       28063   121936      7.0      4280
98557  317376       28063   121936      8.0      2724
98558  317377       28063   121936      9.0      4019
98559  317378       28063   121936     10.0     42731
98560  317379       28063   121936     11.0      4148
98561  317380       28063   121936     12.0     44020
98562  317381       28063   121936     13.0     V4581
98563  317382       28063   121936     14.0     25000
98564  317383       28063   121936     15.0      5997
98565  317384       28063   121936     16.0     E8796


In [139]:
print(df_notes_diabetes[(df_notes_diabetes['HADM_ID'] == 121936)])

     ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE            CHARTTIME  \
0       184       28063  121936.0  2125-02-16                  NaN   
1     59801       28063  121936.0  2125-02-12                  NaN   
2     59802       28063  121936.0  2125-02-11                  NaN   
3    105548       28063  121936.0  2125-02-10                  NaN   
4    105549       28063  121936.0  2125-02-09                  NaN   
5    105501       28063  121936.0  2125-02-16                  NaN   
6    105502       28063  121936.0  2125-02-13                  NaN   
7    105503       28063  121936.0  2125-02-12                  NaN   
8    105504       28063  121936.0  2125-02-11                  NaN   
9    105547       28063  121936.0  2125-02-10                  NaN   
10   999237       28063  121936.0  2125-02-09  2125-02-09 21:01:00   
11   999458       28063  121936.0  2125-02-11  2125-02-11 14:42:00   
12   999719       28063  121936.0  2125-02-13  2125-02-13 08:04:00   
13   999539       28

In [141]:
print(len(df_diag_diabetes['ICD9_CODE'].unique()))

4103


In [143]:
df_diag_diabetes['ICD9_CODE_ROLLED'] = df_diag_diabetes['ICD9_CODE'].apply(lambda x: x[0:3])
print(len(df_diag_diabetes['ICD9_CODE_ROLLED'].unique()))

781


In [145]:
print(temp['NTOKENS_LEN'].sum() / len(temp['NTOKENS_LEN']))

309.62789487249427
